# Introduction

In [ ]:
from powerbi_widget import Report

In [ ]:
import requests

# Embed config for sample report
res = requests.get('https://playgroundbe-bck-1.azurewebsites.net/Reports/SampleReport')

access_token = res.json()['EmbedToken']['Token']
embed_url = res.json()['EmbedUrl']

In [ ]:
report = Report(access_token, embed_url, token_type=1)

In [ ]:
# Render report
report

In [ ]:
# Change dimensions of embedded report
report.set_dimensions(300, 450)

Extract data feature

In [ ]:
page_name = 'ReportSection600dd9293d71ade01765'
visual_name = 'VisualContainer1'

exported_data_task = report.extract_data(page_name, visual_name, rows=5)

# This doesn't work (blocks the kernel indefinitely)
# exported_data = await report._extract_data(page_name, visual_name, rows=20)

In [ ]:
# Get the result after task is finished
exported_data = exported_data_task.result()

print(exported_data)

In [ ]:
page_name = 'ReportSection600dd9293d71ade01765'
visual_name = 'VisualContainer1'

exported_data_task2 = report.extract_data(page_name, visual_name, rows=2)

In [ ]:
exported_data_task2.result()